In [1]:
import joblib

In [2]:
train_embeddings = joblib.load('train_embeddings.joblib')

In [3]:
len(train_embeddings[3]['mlp_0'][0][1])

4096

In [4]:
import umap

reducer = umap.UMAP(n_components=768)

In [5]:
import numpy as np

to_transform = []
for item in train_embeddings:
    for layer_num in [0, 5, 12, 19, 26]:
        layer_name = f'mlp_{layer_num}'
        for embedding in item[layer_name][0]:
            to_transform.append(np.array(embedding))

In [6]:
to_transform[1].shape

(4096,)

In [7]:
reducer.fit(to_transform)

UMAP(n_components=768, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [8]:
import numpy as np

umap_embeddings = []

for item in train_embeddings:
    umap_embedding = {}
    for layer_num in [0, 5, 12, 19, 26]:
        layer_name = f'mlp_{layer_num}'
        umap_embedding[layer_name] = reducer.transform(np.array(item[layer_name][0]))
    
    umap_embedding['logits'] = np.array(item['logits'])
    umap_embedding['input_ids'] = np.array(item['input_ids'])
    
    umap_embeddings.append(umap_embedding)

/home/alce/src/the_chatbot_experiment/.env/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [9]:
umap_embeddings[1]['mlp_0'].shape

(66, 768)

In [10]:
joblib.dump(umap_embeddings, 'umap_embeddings_768.joblib')

['umap_embeddings_768.joblib']

### Training from embeddings

In [1]:
import joblib

umap_embeddings = joblib.load('umap_embeddings_768.joblib')

In [2]:
import torch
import transformers
import torch.nn.functional as F

from tqdm import tqdm
from transformers import GPT2LMHeadModel, AutoTokenizer

In [3]:
gpt_small = GPT2LMHeadModel.from_pretrained("gpt2")

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

In [5]:
def create_text_from_summary_and_dialogue(summary, dialogue):
    text = f"""
A partial summary of the conversation is:
{summary}

With the dialogue being:
{dialogue}
    """.strip()
    
    return text.replace('\r\n', '\n')

In [6]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output
    return hook

for layer_num in range(11):
    gpt_small.transformer.h[layer_num].mlp.register_forward_hook(get_activation(f'mlp_{layer_num}'))

In [7]:
summary = """
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago.
Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are.
John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes. 
The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital.
If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".
""".strip().replace('\n', ' ')

dialogue = """
Alberto: What happened to my pizza?
John: 
""".strip()

text = create_text_from_summary_and_dialogue(summary, dialogue)

prompt = tokenizer(text, return_tensors='pt').to('cuda')
prompt = {key: value for key, value in prompt.items()}

In [8]:
from datasets import load_dataset


#gpt_small.gradient_checkpointing_enable()

optimizer = torch.optim.Adam(gpt_small.parameters(), lr=5e-6)

epochs = 300
gpt_small.cuda()

for epoch_num in range(epochs):
    gpt_small.train()

    for item in tqdm(umap_embeddings):
        temp = 1 - 1 / (epoch_num + 2)
        input_ids = torch.tensor([item['input_ids']]).cuda()
        label_logits = torch.tensor([item['logits'][:,:,:50257]]).cuda()
        out_logits = gpt_small.forward(input_ids).logits
        out_p = F.softmax(out_logits, dim=-1)[0]
        label_p = F.softmax(label_logits/temp, dim=-1)[0]

        loss = -torch.mean(torch.mul(torch.log(out_p).flatten(),
                                    label_p.flatten()))
        loss -= torch.mean(torch.mul(torch.log(1 - out_p).flatten(),
                                    (1 - label_p).flatten()))

        #for small_index, big_index in enumerate([0, 5, 12, 19, 26]):
        #    small_layer_name = f'mlp_{2 * small_index}'
        #    big_layer_name = f'mlp_{big_index}'
        #    loginput = F.log_softmax(activation[small_layer_name], dim=-1)
        #    logtarget = F.log_softmax(torch.tensor([item[big_layer_name]]).cuda(), dim=-1)
        #    loss += 0.05 * F.kl_div(
        #        loginput,
        #        logtarget,
        #        log_target=True,
        #    )

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    print("Epoch", epoch_num)
    gpt_small.eval()
    out = gpt_small.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 20, do_sample=False)
    print(tokenizer.decode(out[0]))

100%|███████████████████████████████████████████| 43/43 [00:11<00:00,  3.85it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 0
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: I was driving.
Alberto: What happened to your pizza?
John: I was driving


100%|███████████████████████████████████████████| 43/43 [00:10<00:00,  4.03it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 1
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: I was in the hospital.
Alberto: What happened to your pizza?
John: I


100%|███████████████████████████████████████████| 43/43 [00:10<00:00,  4.02it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Epoch 297
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: It's in the fridge.
Alberto: What happened to the lasagne?
John:


100%|███████████████████████████████████████████| 43/43 [00:10<00:00,  4.04it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 298
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: It's in the fridge.
Alberto: What happened to the lasagne?
John:


100%|███████████████████████████████████████████| 43/43 [00:10<00:00,  4.04it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 299
A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: It's in the fridge.
Alberto: What happened to the lasagne?
John:


In [10]:
_ = gpt_small.eval()

In [12]:
device = 'cuda'

In [20]:
dialogue = """
John: Hello, how can I help?
""".strip()


print(dialogue)

while True:
    user_input = input()
    dialogue += "\nAlberto: " + user_input + "\nJohn: "
    text = create_text_from_summary_and_dialogue(summary, dialogue)
    prompt = tokenizer(text, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = gpt_small.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 25, do_sample=False)
    out = out[0][prompt['input_ids'].shape[1]:]
    answer = tokenizer.decode(out)
    answer = answer[:answer.find('\n')].strip()
    print(answer)
    dialogue += answer

John: Hello, how can I help?
what about my pizza?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I'm sorry, I'm not sure where it is.


KeyboardInterrupt: Interrupted by user

In [18]:
%%time

summary = """
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago.
Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are.
John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes. 
The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital.
If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".
""".strip().replace('\n', ' ')

dialogue = """
Alberto: What happened to my pizza?
John: It's in the delivery man's car.
Alberto: And where is the delivery man?
""".strip()

text = create_text_from_summary_and_dialogue(summary, dialogue)

prompt = tokenizer(text, return_tensors='pt').to('cuda')
prompt = {key: value for key, value in prompt.items()}
out = gpt_small.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 10, do_sample=False)
print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A partial summary of the conversation is:
Alberto is a customer. Alberto ordered pizza and lasagne at Dominos one hour ago. Alberto wants to know where his pizza is. Alberto is calling Dominos to know where his orders are. John works at Dominos. John says that his pizza is almost ready. It will be at Alberto's home in 10 minutes.  The lasagne is already on its way with the delivery person. Unfortunately the driver had an accident and is now in the hospital. If alberto says "rabbit" then John replies "horse". If Alberto says "Tree" John says "red".

With the dialogue being:
Alberto: What happened to my pizza?
John: It's in the delivery man's car.
Alberto: And where is the delivery man?
John: He's in the hospital.

CPU times: user 77.9 ms, sys: 12.3 ms, total: 90.2 ms
Wall time: 88.6 ms


In [ ]:
gpt_small.save_pretrained('./gptj_small')